#### Libraries

In [ ]:
import pandas as pd
import numpy as np
import os

import openpyxl

import seaborn as sns
import matplotlib.pyplot as plt

#Plotly Tools
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.offline as offline
offline.init_notebook_mode()
from plotly import tools
import plotly.tools as tls
init_notebook_mode(connected=True)


import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)


# Max column
pd.set_option('display.max_columns', None)

In [ ]:
#Location of file
os.getcwd()

#### Load Data

* List of purchaser

In [ ]:
#ID file location
ID_file = pd.read_excel("D:/Shaikh_Desktop_Mydoc/Desktop/Automation Projects/20230727_Savings_Report_Commercial/Data/ID2.xlsx")

# Extract the IDs from the ID file
IDs = ID_file['Name']

* Monthly data

In [ ]:
file_path = "D:/Shaikh_Desktop_Mydoc/Desktop/Automation Projects/20230727_Savings_Report_Commercial/Data/August_23.xls"

In [ ]:
# Define the data types for specific columns
data_types = {
    'Order Date': 'datetime64',
    'Item Code': str,
    'POReference': str,
    'Supplier Id': str
}

# Load the data from Excel file, skipping the first 5 rows, and applying data types
MonthlyData = pd.read_excel(file_path, skiprows=range(0, 5), dtype=data_types)

# Print the DataFrame
display(MonthlyData.head(5))
display(MonthlyData.info())
display(MonthlyData.shape)


In [ ]:
MonthlyData.columns

In [ ]:
# Specify the columns you want to keep from the other file
columns_to_keep = ['Unnamed: 0', 'Business', 'Unnamed: 2', 'Supplier Id', 'Supplier Name',
       'Unnamed: 5', 'Unnamed: 6', 'Country', 'Unnamed: 8', 'Currency',
       'POReference', 'Unnamed: 11', 'Order Date', 'Order Type', 'Category',
       'Item Code', 'Item Name', 'Unit', 'POQuantity', 'PORate', 'Amount',
       'Advance', 'Create By', 'Create Date']

In [ ]:
# Filter the other file based on the IDs and select the desired columns
MonthlyData_Commercial_only = MonthlyData[MonthlyData['Create By'].isin(IDs)][columns_to_keep]

MonthlyData_Commercial_only
display(MonthlyData_Commercial_only.shape)

In [ ]:
MonthlyData['Business'].unique()

## Data cleaning and Data manipulation

<p> Remove Order type 'Biz Promo', 'Contract Order' </p>
<p> Remove Business ACI CO-RO </p>
<p> <mark> Remove row if Category contains PI </mark> </p>
<p> <mark> Remove 'FG' and 'RM' from Electrical </mark> </p>
<p> Remove Colgate, Cevac, Freight, Colgate </p>
<p> Remove Import items from 'Vegetables', 'Field Crops', 'Vegetables-EWS'</p>

In [ ]:
##________________________________ the values to remove________________________________

values_to_remove_ordertype = ['Biz Promo', 'Contract Order']
values_to_remove_business = ['ACI CO-RO']
values_to_remove_Category = ['PI']

# Filter the DataFrame to remove rows where "Order Type" column contains specific values
df = MonthlyData_Commercial_only[~MonthlyData_Commercial_only['Order Type'].isin(values_to_remove_ordertype)]
df = df[~df['Business'].isin(values_to_remove_ordertype)]
df = df[~df['Item Name'].str.contains('Colgate', case=False)]
df = df[~df['Item Name'].str.contains('Cevac', case=False)]
df = df[~df['Item Name'].str.contains('Freight', case=False)]
df = df[~df['Supplier Name'].str.contains('Colgate', case=False)]
df = df[~df['Category'].isin(values_to_remove_Category)]



# ### Define the conditions for removing rows
# business_to_exclude = ['Electrical']
# category_to_exclude = ['FG', 'RM']

# # Filter rows based on the specified conditions
# df = df[~((df['Business'].isin(business_to_exclude)) & (df['Category'].isin(category_to_exclude)))]


### Define the conditions for removing rows
business_to_exclude2 = ['Vegetables', 'Field Crops', 'Vegetables-EWS']
category_to_exclude2 = ['Import']

# Filter rows based on the specified conditions
df = df[~((df['Business'].isin(business_to_exclude2)) & (df['Order Type'].isin(category_to_exclude2)))]







##________________________________ the values to rename________________________________

df.loc[(df['Order Type'] == 'Import') & (df['Currency'] == 'BDT'), 'Order Type'] = 'Native'    # Local LC to Local
df['Order Type'] = df['Order Type'].apply(lambda x: x.replace('Native', 'Local'))              # Native to Local


display(df.shape)




#Note
#The ~ operator negates the condition, so it selects rows where the 


* Outlier detection

In [ ]:
Local_Non_BDT1 = df.loc[(df['Order Type'] == 'Local') & (df['Currency'] != 'BDT')]
Local_Non_BDT2 = df.loc[(df['Order Type'] == 'Native')]
Import_BDT = df.loc[(df['Order Type'] == 'Import') & (df['Currency'] == 'BDT')]

In [ ]:
duplicate_rows = df[df.duplicated(subset=['Currency','Item Code', 'POQuantity'], keep=False)]

### Import Data from Another DataBase

In [ ]:
# File Path for WAP

File_WAP = "D:/Shaikh_Desktop_Mydoc/Desktop/Automation Projects/20230727_Savings_Report_Commercial/Output_file/Lastprice_WAP_AVRate.xlsx"

In [ ]:
# Name of Sheets in WAP file

# Load the Excel file using openpyxl
workbook = openpyxl.load_workbook(File_WAP)

# Get the list of sheet names
sheet_names = workbook.sheetnames

# Print the list of sheet names
display(sheet_names)


* Average Rate

In [ ]:
# Specify the sheet name from which you want to load data
AVRate_fy_22_23 = "AV_Rate" 

# Load data from the specified sheet
AVRate_ = pd.read_excel(File_WAP, sheet_name= AVRate_fy_22_23)

display(AVRate_.columns)
display(AVRate_.head(5))
display(AVRate_.shape)


# AVRate_.rename(columns={'PORate': 'AVRate'}, inplace=True)


display(AVRate_.columns)
display(AVRate_.head(5))
display(AVRate_.shape)




df = pd.merge(df, AVRate_, on=['Business', 'Item Code', 'Currency'], how='left')
display(df.head(5))
display(df.shape)

* WAP 

In [ ]:
# Specify the sheet name from which you want to load data
WAP_fy_22_23 = "WAP" 

# Load data from the specified sheet
WAP_ = pd.read_excel(File_WAP, sheet_name=WAP_fy_22_23)

display(WAP_.columns)
display(WAP_.head(5))
display(WAP_.shape)

In [ ]:
# left outer join


df_report = pd.merge(df, WAP_, on=['Business', 'Item Code', 'Currency'], how='left')
display(df_report.head(5))
display(df_report.shape)

* WAP: 6 Months 

In [ ]:
# Specify the sheet name from which you want to load data
WAP_6M = "6M_WAP" 

# Load data from the specified sheet
WAP_6M_ = pd.read_excel(File_WAP, sheet_name=WAP_6M)

display(WAP_6M_.columns)
display(WAP_6M_.head(5))
display(WAP_6M_.shape)

In [ ]:
# left outer join


df_report = pd.merge(df_report, WAP_6M_, on=['Business', 'Item Code', 'Currency'], how='left')
display(df_report.head(5))
display(df_report.shape)

* June or previous

In [ ]:


# Specify the sheet name from which you want to load data
July_or_previous = "July23" 

# Load data from the specified sheet
July__ = pd.read_excel(File_WAP, sheet_name=July_or_previous)

display(July__.columns)
display(July__.head(5))
display(July__.shape)



# Columns for June or previous 

columns_to_keep_June = ['Business','Currency','Item Code', 'PORate']
July__ = July__[columns_to_keep_June]

July__.rename(columns={'PORate': 'June or Previous price'}, inplace=True)

July__.columns
display(July__.head(5))
display(July__.info())

In [ ]:
# left outer join


df_report = pd.merge(df_report, July__, on=['Business', 'Item Code', 'Currency'], how='left')
display(df_report.head(5))
display(df_report.shape)

* Last Price

In [ ]:
data_types = {
    'Order Date': 'datetime64',
    'Item Code': str,
    'POReference': str,
    'Supplier Id': str
}




# Specify the sheet name from which you want to load data
Last_Price = "Lastprice" 

# Load data from the specified sheet
Last_Price_ = pd.read_excel(File_WAP, sheet_name= Last_Price, dtype = data_types)

display(Last_Price_.columns)
display(Last_Price_.head(5))
display(Last_Price_.shape)



# Columns for June or previous 

columns_to_keep_LP = ['Business','Currency','POReference','Item Code', 'Last_Price']
Last_Price__ = Last_Price_[columns_to_keep_LP]

# Last_Price__.rename(columns={'PORate': 'LastPrice'}, inplace=True)

Last_Price__.columns
display(Last_Price__.head(5))
display(Last_Price__.info())

In [ ]:
# left outer join


df_report = pd.merge(df_report, Last_Price__, on=['POReference', 'Business', 'Item Code', 'Currency'], how='left')
display(df_report.head(5))
display(df_report.shape)

In [ ]:
df_report_ = df_report.copy()

### Analytics : 001

* Calculate variance (WAP vs Average Rate)

In [ ]:
df_Analytics = df_report.copy()
display(df_Analytics.columns)


In [ ]:

columns_to_keep_Analytics = ['Business', 'Supplier Id', 'Supplier Name','Country', 'Currency','POReference', 'Order Date', 'Order Type', 'Category',
'Item Code', 'Item Name', 'Unit', 'POQuantity', 'PORate', 'Amount','Advance', 'Create By', 'Create Date', 'AVRate', 'WAP',
       'June or Previous price', 'Last_Price']
df_Analytics = df_Analytics[columns_to_keep_Analytics]

display(df_Analytics.head(5))
display(df_Analytics.shape)
display(df_Analytics.info())

In [ ]:
# Calculate the variance between AVRate and WAP
df_Analytics['Difference'] = (df_Analytics['AVRate'] - df_Analytics['WAP'])
df_Analytics['Variance_P'] = ((df_Analytics['AVRate'] - df_Analytics['WAP'])/df_Analytics['AVRate'])*100

# Create a separate DataFrame for high variance cases
high_variance_df = df_Analytics[df_Analytics['Variance_P'].abs() >5]

# Print the DataFrames
display("Original DataFrame:")
display(df_Analytics.head())
display("\nHigh Variance DataFrame:")
display(high_variance_df.head())
display(high_variance_df.shape)

In [ ]:
display(high_variance_df.head(5))
display(high_variance_df.shape)

### Load data: From historical DB

In [ ]:
# Folder path containing the files
pathSpecified = "D:/Shaikh_Desktop_Mydoc/Desktop/Automation Projects/20220622 PriceMirror/MMS2018JanTo2023Jun"  # Replace with the actual folder path

# Using listdir() function  
listOfFileNames = os.listdir(pathSpecified)


# Print the name of all files in directory  
print("Following is the list of names of all the files present in the specified directory: ")  
# Create an empty DataFrame to store the combined data
combined_data = pd.DataFrame()
print(listOfFileNames)



# Create an empty DataFrame to store the combined data
combined_data = pd.DataFrame()


# Iterate over the file names
for file_name in listOfFileNames:
    # Construct the file path
    file_path = os.path.join(pathSpecified, file_name)
    # Read the file into a DataFrame
    # Define the skiprows and column data types
    skiprows = 5
    dtype = {
        'Order Date': 'datetime64',
        'Item Code': str,
        'POReference': str,
        'Supplier Id' : str,
        
        
    }

    # Load the Excel file
    data = pd.read_excel(file_path, index_col=False, skiprows=range(0, 5), dtype=dtype).iloc[0:].reset_index(drop=True)
    combined_data = combined_data.append(data)
# Display the loaded data
combined_data.head(4)

In [ ]:
# business_to_keep = ['ACI Animal Genetics',
# 'ACI Consumer Electronics',
# 'ACI Crop Care (ACI Formulations Limited)',
# 'ACI Flour',
# 'ACI Foods Ltd.',
# 'ACI Premio Plastics',
# 'Aerosol',
# 'Animal Health',
# 'CB Core ', 
# 'Edible Oil',
# 'Fertilizer',
# 'Hygiene',
# 'Paint',
# 'Pharma',
# 'Premiaflex',
# 'ACI Salt',
# 'Electrical',
# 'Field Crops',
# 'Mosquito Coil',
# 'Rice',
# 'Vanish',
# 'Vegetables',
# 'Vegetables-EWS',
# ]

# # business_to_keep =['Hygiene']


# combined_data = combined_data[combined_data['Business'].isin(business_to_keep)]

### Analytics :002

In [ ]:
combined_data['Year'] = combined_data['Order Date'].dt.year
combined_data['Month'] = combined_data['Order Date'].dt.month_name()

import pandas as pd

def calculate_wap(amount, quantity):
    if quantity != 0:
        wap = amount / quantity
        return wap
    else:
        return None

# Group by 'Item Code', 'Year', and 'Month', and calculate the WAP
grouped_data = combined_data.groupby(['Order Type','Business','Item Code','Currency', 'Year', 'Month']).apply(lambda x: calculate_wap(x['Amount'].sum(), x['POQuantity'].sum()))

# Reset the index and rename the column
grouped_data = grouped_data.reset_index().rename(columns={0: 'WAP_MOM'})

grouped_data['Day'] = 1



# Generate automatic column names
column_names = grouped_data['Year'].astype(str) + '-' + grouped_data['Month'].astype(str) + '-' + grouped_data['Day'].astype(str)

pivot_df = grouped_data.pivot_table(index=['Business', 'Item Code','Currency'], columns=column_names, values='WAP_MOM', aggfunc=lambda x: x).reset_index()


# Define the desired column order
desired_order = ['Business', 'Item Code', 'Currency']

# Extract date columns and sort them in chronological order
date_columns = sorted([col for col in pivot_df.columns if '-' in col], key=lambda x: pd.to_datetime(x, format='%Y-%B-%d'))

# Concatenate the desired order with the sorted date columns
desired_order += date_columns

# Reindex the DataFrame columns
pivot_df = pivot_df.reindex(columns=desired_order)

display(pivot_df.head(5))

In [ ]:
pivot_df.columns

In [ ]:
columns_to_keep_for_wap = ['Business', 'Item Code', 'Currency', '2022-January-1', '2022-February-1', '2022-March-1', '2022-April-1',
       '2022-May-1', '2022-June-1', '2022-July-1', '2022-August-1',
       '2022-September-1', '2022-October-1', '2022-November-1',
       '2022-December-1', '2023-January-1', '2023-February-1', '2023-March-1',
       '2023-April-1', '2023-May-1', '2023-June-1', '2023-July-1']

pivot_df_wap = pivot_df[columns_to_keep_for_wap]

pivot_df_wap.head(5)

In [ ]:
# left outer join


high_variance_df_ANA2 = pd.merge(high_variance_df, pivot_df_wap, on=['Business', 'Item Code', 'Currency'], how='left')
display(high_variance_df_ANA2.head(5))
display(high_variance_df_ANA2.shape)

### Make Business group 

In [ ]:
# Function to map business values
def map_business(business):
    if business in ['ACI Flour', 'ACI Foods Ltd.', 'ACI Salt', 'Edible Oil']:
        return 'Foods,Flour,Salt& Edible Oils'
    elif business == 'ACI Consumer Electronics':
        return 'ACI Consumer Electronics'
    elif business == 'ACI Crop Care (ACI Formulations Limited)':
        return 'ACI Crop Care'
    elif business == 'ACI Premio Plastics':
        return 'ACI Premio Plastics'
    elif business == 'Animal Health':
        return 'Animal Health'
    elif business == 'CB Core ':
        return 'Consumer'
    elif business == 'Pharma':
        return 'Consumer'
    elif business == 'Electrical':
        return 'Electrical'
    elif business in ['Fertilizer', 'Field Crops', 'Vegetables', 'Vegetables-EWS']:
        return 'Fertilizer'
    elif business == 'Hygiene':
        return 'Hygiene'
    elif business == 'Paint':
        return 'Paint'
    elif business == 'ACI Animal Genetics':
        return 'ACI Animal Genetics'
    elif business == 'Premiaflex':
        return 'Premiaflex'
    elif business == 'Rice':
        return 'Rice'
    elif business in ['Aerosol', 'Vanish', 'Mortin']:
        return 'SCJ'
    else:
        return "Other"

# Apply the mapping function to the 'Business' column
df_report_['Business Name'] = df_report_['Business'].apply(map_business)

display(df_report_.head())
display(df_report_.columns)

In [ ]:
grouped = df_report_.groupby(['Business', 'Business Name'])['Amount'].sum().reset_index()
display(grouped.sort_values(by='Business Name'))

### Procurement & Savings Claculation 

* Conversion rate

In [ ]:
display(df_report_["Currency"].unique())

Value_of_USD = 110
Value_of_EUR = 120
Value_of_GBP = 135
Value_of_BDT = 1

In [ ]:
def Total_ampunt_cal (row):
    if row['Currency']=='USD': 
        return row['POQuantity']* row['PORate']*Value_of_USD
    elif row['Currency']=='EUR': 
        return row['POQuantity']* row['PORate']*Value_of_EUR
    elif row['Currency']=='GBP': 
        return row['POQuantity']* row['PORate']*Value_of_GBP
    elif row['Currency']=='BDT': 
        return row['POQuantity']* row['PORate']*Value_of_BDT
    else:
        return "No Currency Match"



def Savings_Jun_cal (row):
    if row['Currency']=='USD': 
        return row['POQuantity']*(row['June or Previous price'] - row['PORate'])*Value_of_USD
    elif row['Currency']=='EUR': 
        return row['POQuantity']* (row['June or Previous price'] - row['PORate'])*Value_of_EUR
    elif row['Currency']=='GBP': 
        return row['POQuantity']* (row['June or Previous price'] - row['PORate'])*Value_of_GBP
    elif row['Currency']=='BDT': 
        return row['POQuantity']* (row['June or Previous price'] - row['PORate'])*Value_of_BDT
    else:
        return "No Currency Match"


def Savings_WAP_cal (row):
    if row['Currency']=='USD': 
        return row['POQuantity']*(row['WAP'] - row['PORate'])*Value_of_USD
    elif row['Currency']=='EUR': 
        return row['POQuantity']* (row['WAP'] - row['PORate'])*Value_of_EUR
    elif row['Currency']=='GBP': 
        return row['POQuantity']* (row['WAP'] - row['PORate'])*Value_of_GBP
    elif row['Currency']=='BDT': 
        return row['POQuantity']* (row['WAP'] - row['PORate'])*Value_of_BDT
    else:
        return "No Currency Match"


    

def Savings_WAP_6M_cal (row):
    if row['Currency']=='USD': 
        return row['POQuantity']*(row['Last_Price'] - row['PORate'])*Value_of_USD
    elif row['Currency']=='EUR': 
        return row['POQuantity']* (row['Last_Price'] - row['PORate'])*Value_of_EUR
    elif row['Currency']=='GBP': 
        return row['POQuantity']* (row['Last_Price'] - row['PORate'])*Value_of_GBP
    elif row['Currency']=='BDT': 
        return row['POQuantity']* (row['Last_Price'] - row['PORate'])*Value_of_BDT
    else:
        return "No Currency Match"

    


def Savings_LastPrice_cal (row):
    if row['Currency']=='USD': 
        return row['POQuantity']*(row['WAP_6M'] - row['PORate'])*Value_of_USD
    elif row['Currency']=='EUR': 
        return row['POQuantity']* (row['WAP_6M'] - row['PORate'])*Value_of_EUR
    elif row['Currency']=='GBP': 
        return row['POQuantity']* (row['WAP_6M'] - row['PORate'])*Value_of_GBP
    elif row['Currency']=='BDT': 
        return row['POQuantity']* (row['WAP_6M'] - row['PORate'])*Value_of_BDT
    else:
        return "No Currency Match"

    


df_report_["Total_Value_in_Taka"] = df_report_.apply(Total_ampunt_cal, axis=1)
df_report_["Savings_June"] = df_report_.apply(Savings_Jun_cal, axis=1)
df_report_["Savings_WAP"] = df_report_.apply(Savings_WAP_cal, axis=1)
df_report_["Savings_WAP_6M"] = df_report_.apply(Savings_WAP_6M_cal, axis=1)
df_report_["Savings_LastPrice"] = df_report_.apply(Savings_LastPrice_cal, axis=1)

display(df_report_.head())
display(df_report_.shape)

### Report

In [ ]:
# Group by "Business Name" and "Local/Import," then apply aggregation
Final_report = df_report_.groupby(['Business Name', 'Order Type']).agg(Total_Value_in_Taka=('Total_Value_in_Taka', 'sum'),
    Savings_June=('Savings_June', 'sum'),
    Savings_WAP=('Savings_WAP', 'sum'),
    Savings_LastPrice=('Savings_LastPrice', 'sum'),
    Savings_WAP_6M=('Savings_WAP_6M', 'sum')).reset_index()

# Display the grouped report
display(Final_report)

In [ ]:
def separate_gain_loss(dataframe, column_name):
    gain_column = "Gain_" + column_name
    loss_column = "Loss_" + column_name
    
    gain_df = dataframe[dataframe[column_name] >= 0]
    loss_df = dataframe[dataframe[column_name] < 0]
    
    return gain_df, loss_df

In [ ]:
# Call the function for each column
gain_june_df, loss_june_df = separate_gain_loss(Final_report, 'Savings_June')
gain_wap_df, loss_wap_df = separate_gain_loss(Final_report, 'Savings_WAP')
gain_wap_6m_df, loss_wap_6m_df = separate_gain_loss(Final_report, 'Savings_WAP_6M')

# Display the Gain and Loss dataframes for each column
print("Gain Data for Savings_June:")
display(gain_june_df)

print("\nLoss Data for Savings_June:")
display(loss_june_df)

print("\nGain Data for Savings_WAP:")
display(gain_wap_df)

print("\nLoss Data for Savings_WAP:")
display(loss_wap_df)

print("\nGain Data for Savings_WAP_6M:")
display(gain_wap_6m_df)

print("\nLoss Data for Savings_WAP_6M:")
display(loss_wap_6m_df)

In [ ]:
# Calculate the correlation matrix
corr_matrix = Final_report.corr()

# Create a correlation heatmap plot
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Heatmap')
plt.show()


In [ ]:
Final_report['Total_Value_in_Taka'].iplot(kind='bar', xTitle='claps',
                  yTitle='count', title='Claps Distribution')

In [ ]:
import plotly           #(version 4.5.4) #pip install plotly==4.5.4
import plotly.express as px
import plotly.io as pio

In [ ]:
barchart = px.bar(
    data_frame=Final_report,
    x="Business Name",
    y="Total_Value_in_Taka",
    color="Order Type",               # differentiate color of marks
    opacity=0.9,                  # set opacity of markers (from 0 to 1)
    orientation="v",              # 'v','h': orientation of the marks
    barmode='relative',
    #hover_name='under_trial',   # values appear in bold in the hover tooltip
     hover_data=['Savings_WAP'],    # values appear as extra data in the hover tooltip
    # custom_data=['others']


)

In [ ]:
pio.show(barchart)

#### Output

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
# path = 
writer = pd.ExcelWriter(r'D:\Shaikh_Desktop_Mydoc\Desktop\Automation Projects\20230727_Savings_Report_Commercial\Output_file\sav_V04.xlsx', engine='xlsxwriter')


# Write the MonthlyData DataFrame to a worksheet.

Final_report.to_excel(writer, sheet_name='Top', index=False)
df_report_.to_excel(writer, sheet_name='COM', index=False)
high_variance_df_ANA2.to_excel(writer, sheet_name='COM_ANA', index=False)
duplicate_rows.to_excel(writer, sheet_name='DuplicateData', index=False)



# Save the writer and close it.
writer.save()
writer.close()